In [1]:
import openai
import json
import time
import sys
import os
from datetime import datetime
from pathlib import Path

In [13]:
# ==== Settings ====

enableVerboseOutput = True
enable_context = True

context_file = "context/context_websearch.txt"
# context_file = "context/context_pseudocode.txt"

In [15]:
# ==== Functions ====

def load_parameters(node_config):
    global api_key
    api_key = node_config['settings'][0]['api_key']

    global model_name
    model_name = node_config['settings'][0]['model_name']
    global max_tokens
    max_tokens = int(node_config['settings'][0]['max_tokens'])
    global temperature
    temperature = float(node_config['settings'][0]['temperature'])
    global top_p
    top_p = float(node_config['settings'][0]['top_p'])
    global frequency_penalty
    frequency_penalty = float(node_config['settings'][0]['frequency_penalty'])
    global vecstore_path
    vecstore_path = node_config['settings'][0]['vectorstore_path']
    global doc_context_size
    doc_context_size = int(node_config['settings'][0]['document_context_size'])
    global dump_save_path
    dump_save_path = node_config['settings'][0]['dump_save_path']

    # Assemble the LLM input config
    global llm_config
    llm_config = {
                    "temp": temperature,
                    "model": model_name,
                    "top_p": top_p,
                    "freq_penalty": frequency_penalty,
                    "max_tokens": max_tokens
                }

def gpt_inference_direct(question_input: str, config):

    if question_input != "":

        if enableVerboseOutput:
            print("Running inference...\n")

        response = openai.chat.completions.create(model=config['model'],
                                                max_tokens=config['max_tokens'],
                                                temperature=config['temp'],
                                                top_p=config['top_p'],
                                                frequency_penalty=config['freq_penalty'],
                                                messages=[
                                                            {"role": "user", "content": question_input}
                                                        ])

        response_str = response.choices[0].message.content
        tokens_used = int(response.usage.total_tokens)

        return response_str, tokens_used

In [11]:
# ==== Load configuration file ====

f = open("config.json",)
config_file = json.load(f)
f.close()

load_parameters(config_file)

# Set-up OpenAI API Key
openai.api_key = api_key
os.environ['OPENAI_API_KEY'] = api_key

In [5]:
# ==== Load context ====

prompt_context = ""

# -> Load file content as instructions and append it to dynamic prompt input
# Check if file is present
if Path(context_file).is_file():
    # File existing
    tmp_file_content = ""
    with open(context_file, encoding='utf8', mode='r') as f:
        for line in f:
            tmp_file_content += f"{line.strip()}\n"
    prompt_context = tmp_file_content

    enable_context = True
    print("--> Context loaded and will be applied on each prompt.")

else:
    print("File not found.")

--> Context loaded and will be applied on each prompt.


In [16]:
# ==== LLM inference ====

# =============================================================

input_prompt = "What's the world's tallest building in 2023?"

# input_prompt = "The Apple Watch is not visible on my new iPhone and I cannot connect it. How do I configure it for the new phone?"

# input_prompt = "My coffee machine has stopped working, no water comes out. Maybe it needs cleaning? But I don't know how to do that."

# input_prompt = "Please make a research about the CO2 footprint of battery electric vehicles and compare with an equivalent vehicle with a modern Diesel engine."

# =============================================================


text_out = ""

# Append context if activated
if enable_context:
    prompt = prompt_context + "\n" + input_prompt

# Inference
text_out, tokens_used_tmp = gpt_inference_direct(prompt, llm_config)
last_prompt = prompt
last_response = text_out

print("GPT >> ")
print(text_out)

if enableVerboseOutput:
    print(" ")
    print(f"--> Token used for request: {tokens_used_tmp}")


Running inference...

GPT >> 
1: world's tallest building 2023
2: current tallest skyscraper 2023
3: highest building in the world as of 2023
 
--> Token used for request: 560
